# Predicting cattle T cell Epitopes in M.bovis

This is a simplified version of an analysis we used to predict cattle T cell epitopes in the proteome of M.bovis. We apply three binding prediction methods to the M. bovis proteome using a subset of human MHC-II alleles to approximate the bovine immune response. Two different strategies are then applied to filter the resulting set of binders: global binder ranking and epitope density. Both these sets were used to make 20mer peptides for synthesis. 

Several other metrics are applied to produce a final list of peptide candidates.

## References

* Integrated computational prediction and experimental validation identifies promiscuous T cell epitopes in the proteome of pathogenic mycobacteria. Damien Farrell, Gareth Jones, Chris Pirson, Kerri M Malone, Kevin Rue-Albrecht, Anthony J Chubb, H Martin Vordermeier, Stephen V. Gordon. 2016
* M. Pandya, “Definition of Bovine Leukocyte Antigen Diversity and Peptide Binding Profiles for Epitope Discovery,” 2016.
* O. T. Schubert et al., “Absolute Proteome Composition and Dynamics during Dormancy and Resuscitation of Mycobacterium tuberculosis,” Cell Host Microbe, 2015.

In [1]:
import os, math, time, pickle, subprocess
from importlib import reload
from collections import OrderedDict
import numpy as np
import pandas as pd
pd.set_option('display.width', 100)
import epitopepredict as ep
from epitopepredict import base, sequtils, plotting, peptutils, analysis
from IPython.display import display, HTML, Image
%matplotlib inline
import matplotlib as mpl
import pylab as plt

##  Alleles
* 8 bovine similar HLA MHC-II alleles were used to approximate cattle responses
* BoLA class I alleles used here are those most frequent in 81 Holstein cattle from the UVM Dairy Center of Excellence research herd assayed by PCR-SSP. (Different alleles were used in the original study).

In [2]:
mhc2alleles = ep.get_preset_alleles('bovine_like_mhc2')
mhc1alleles = ['BoLA-1:01901', 'BoLA-2:00801','BoLA-2:01201','BoLA-4:02401','BoLA-2:07001',
               'BoLA-3:01701','BoLA-1:02301','BoLA-2:01801','BoLA-6:01302','BoLA-3:00201']

## Filtering with MTB srm data

Here we load the protein sequences from a genbank file. MTB is used since it is 99% similar to Bovis. You can find this file at https://www.ncbi.nlm.nih.gov/genome/166?genome_assembly_id=159857

The proteins are filtered using absolute abundance levels of proteins in an unfractionated mixed lysate of M. tuberculosis H37Rv cultures identified by Schubert et al. All proteins undetected using selected reaction monitoring (SRM) (concentration = 0 and <2 peptides detected) in this study were filtered out. This leaves 1870 out of ~4000 proteins.

In [3]:
#srm data
srm = pd.read_csv('srm_mtb.csv')
#srm = srm[srm.length<=400]
proteome = sequtils.genbank_to_dataframe('MTB-H37Rv.gb',cds=True)
#filter proteins on srm data
proteome = proteome.merge(srm[['locus_tag','concentration']],on='locus_tag',how='inner')
proteome = proteome[proteome.concentration>0]
print (len(proteome))
proteome.head()

1870


,type,protein_id,locus_tag,gene,db_xref,product,note,translation,pseudo,start,end,strand,length,order,concentration
0,CDS,CCP42723.1,Rv0001,dnaA,GI:444893470,Chromosomal replication initiator protein DnaA,"Rv0001, (MT0001, MTV029.01, P49993), len: 507 ...",MTDDPGSGFTTVWNAVVSELNGDPKVDDGPSSDANLSAPLTPQQRA...,NaN,0,1524,1,507,1,1
1,CDS,CCP42724.1,Rv0002,dnaN,GI:444893471,DNA polymerase III (beta chain) DnaN (DNA nucl...,"Rv0002, (MTV029.02, MTCY10H4.0), len: 402 aa. ...",MDAATTRVGLTDLTFRLLRESFADAVSWVAKNLPARPAVPVLSGVL...,NaN,2051,3260,1,402,2,9
2,CDS,CCP42725.1,Rv0003,recF,GI:444893472,DNA replication and repair protein RecF (singl...,"Rv0003, (MTCY10H4.01), len: 385 aa. RecF, DNA ...",MYVRHLGLRDFRSWACVDLELHPGRTVFVGPNGYGKTNLIEALWYS...,NaN,3279,4437,1,385,3,1
3,CDS,CCP42727.1,Rv0005,gyrB,GI:444893474,DNA gyrase (subunit B) GyrB (DNA topoisomerase...,"Rv0005, (MTCY10H4.03), len: 675 aa. GyrB, DNA ...",MAAQKKKAQDEYGAASITILEGLEAVRKRPGMYIGSTGERGLHHLI...,NaN,5239,7267,1,675,5,5
4,CDS,CCP42728.1,Rv0006,gyrA,GI:444893475,DNA gyrase (subunit A) GyrA (DNA topoisomerase...,"Rv0006, (MTCY10H4.04), len: 838 aa. GyrA, DNA ...",MTDTTLPPDDSLDRIEPVDIEQEMQRSYIDYAMSVIVGRALPEVRD...,NaN,7301,9818,1,838,6,10


## Predict binders

Run binding predictions for each of the three and save to disk. This enables us to reload the results to re-analyse.

In [27]:
P1 = base.get_predictor('tepitope')
b = P1.predict_sequences(proteome, alleles=mhc2alleles, cpus=8, path='mtb_tepitope')

took 75.966 seconds
predictions done for 1870 sequences in 8 alleles
results saved to /home/damien/gitprojects/epitopepredict/examples/mtb_tepitope


In [20]:
P2 = base.get_predictor('netmhciipan')
b = P2.predict_sequences(proteome, alleles=mhc2alleles, cpus=8, path='mtb_netmhciipan',overwrite=False)

/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)
/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)
/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)
/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)
/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)
/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


took 4978.484 seconds
predictions done for 1870 sequences in 8 alleles
results saved to /home/damien/gitprojects/epitopepredict/examples/mtb_netmhciipan


In [5]:
reload(base)
P3 = base.get_predictor('netmhcpan')
b = P3.predict_sequences(proteome[:200], alleles=mhc1alleles, cpus=8, path='mtb_netmhcpan',overwrite=False)

took 1496.114 seconds
predictions done for 200 sequences in 10 alleles
results saved to /home/damien/gitprojects/epitopepredict/examples/mtb_netmhcpan


## Compute clusters of promiscuous binders

Here we have detected clusters of MHC II binders. These clusters vary in length and for peptides synthesis we must create a list of 20-mers covering shorter sequences or splitting longer sequences into 2. For the 20mers hydrophobicity and net charge are calculated.

In [21]:
reload(analysis)
predictors = ['netmhcpan','tepitope','netmhciipan']
clusters = {}
for name in predictors:
    P = base.get_predictor(name)
    path = 'mtb_'+name
    P.load(path)   
    pb=P.promiscuous_binders(n=3)
    #print (pb[:20])
    cl = analysis.find_clusters(pb, min_binders=3)
    clusters[name] = cl
 

## overlapping clusters?

In [22]:
c1=clusters['netmhciipan']
c2=clusters['tepitope']
x = analysis.get_overlaps(c1,c2,how='inside')
print (x)

         name  start  end  binders  length  overlap
0      Rv0002    175  193        3      18        0
2      Rv0003    300  331        5      31        0
1      Rv0003    153  174        4      21        0
3      Rv0005    472  493        3      21        0
4      Rv0005    544  558        3      14        0
5      Rv0006    311  335        3      24        0
8      Rv0006    433  450        3      17        0
6      Rv0006    369  384        3      15        0
7      Rv0006    386  399        3      13        0
9      Rv0006    565  578        3      13        0
10    Rv0015c      4   22        3      18        0
11    Rv0018c      0   16        3      16        0
12    Rv0019c     18   36        4      18        0
13     Rv0034     82   95        3      13        0
14    Rv0036c    215  230        3      15        0
16    Rv0037c    199  217        3      18        0
17    Rv0037c    274  288        3      14        0
15    Rv0037c     31   44        3      13        0
20     Rv004

In [ ]:
final = ep.analysis.create_nmers(cl, proteome, key='20mer', length=20, margin=2)     
final = analysis.peptide_properties(final, colname='20mer')
print (final.head())   

In [ ]:
reload(plotting)
#print (P.get_names())
ax = plotting.plot_tracks([P],name='Bla-g-5',cutoff=.95, cutoff_method='default',n=2, legend=True, figsize=(12,5),regions=cl)
